In [4]:
from BHDVCS_tf import DvcsData
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

df = pd.read_csv("dvcs_bkm_xs_June2021_4pars.csv")
errValues = [0.0, 0.01, 0.025, 0.05, 0.1, 0.15, 0.25, 0.30]

def getReals(setNum, numPhi = 45):
    df = pd.read_csv("dvcs_bkm_xs_June2021_4pars.csv")
    return df.loc[numPhi * setNum, 'gReH': 'gReHTilde']

def getColorMappingValues(errF, amountofSets):
    ReHMean = []
    ReEMean = []
    ReHtildeMean = []
    for set in range(amountofSets):
        data = pd.read_csv("Predictions\Method2-"+ str(set) + "-" + str(errF) +  ".csv")
        
        # NOTE we take max(mean, -1) because if data is more than 100% error it should just cap in terms of heatmapping
        #Inverted because currently smaller values are red and larger values are green
        #Want to make opposite (smaller %error-values = green and larger %error-values = red) therefore we multiply by -1
        ReHMean.append(100 * max(-abs(np.mean(data.ReH)), -1))
        ReEMean.append(100 * max(-abs(np.mean(data.ReE)), -1))
        ReHtildeMean.append(100 * max(-abs(np.mean(data.ReHtilde)), -1))

    return ReHMean, ReEMean, ReHtildeMean  

def getSizes(errF, amountofSets):
    ReHMean = []
    ReEMean = []
    ReHtildeMean = []
    for set in range(amountofSets):
        data = pd.read_csv("Predictions\Method2-"+ str(set) + "-" + str(errF) +  ".csv")
        
        ReHMean.append(10 + pow(np.std(data.ReH), 0.75))
        ReEMean.append(10 + pow(np.std(data.ReE), 0.75))
        ReHtildeMean.append(10 + pow(np.std(data.ReHtilde), 0.75))

    return ReHMean, ReEMean, ReHtildeMean

#Returns lists that contain the kinematics for each index
def getkinematics(numSets):
    k = []
    QQ = []
    xb = []
    t = []
    for i in range(numSets):
        #gets a set's kinematics
        kvar, Q, x, ts = df.loc[45 * i, 'k': 't']
        
        #adds set's kinematics to running list
        k.append(kvar)
        QQ.append(Q)
        xb.append(x)
        t.append(ts)
    
    return k, QQ, xb, t

def plotInteractive(err, numSets, whichCFF, show = True):
    h, e, tilde = getColorMappingValues(err, numSets)
    hs, es, tildes = getSizes(err, numSets)
    _, QQ, xb, t = getkinematics(numSets)


    chosens = []
    chosen = []
    if whichCFF == 'ReH':
        chosens = hs
        chosen = h
    elif whichCFF == 'ReE':
        chosens = es
        chosen = e
    else: 
        chosen = tilde
        chosens = tildes
        whichCFF = 'ReHtilde'

    # Blue means better predictions -- Red --> Worse Predictions
    splot = go.Figure(data=[go.Scatter3d(
        x=QQ,
        y=t,
        z=xb,
        mode='markers',
        marker=dict(
            showscale=True,
            cmax=0,
            cmin=-100,
            size=chosens,
            color=chosen,                # set color to an array/list of desired values
            colorscale='RdYlGn',   # choose a colorscale
            opacity=0.8
        ),
        hovertemplate = "QQ: %{x:.2f}<br>t: %{y:.2f}<br>xb: %{z:.2f}"
    )])

    splot.update_traces
    splot.update_layout(
        title = 'Prediction Accuracy on ' + whichCFF + ' with ' + str(err*100) + '% Error on F',
        scene = dict(
            xaxis_title='QQ',
            yaxis_title='t',
            zaxis_title='x_b',
            aspectmode='cube',
        ),
        template='ggplot2',
        margin=dict(r=0, b=40, l=0, t=50)
    )

    #splot.write_html('Interactive Plots/Mega/Acc+Spread_' + whichCFF + '_' +str(err*100) + 'ErrF.html')

    if show:
        splot.show()

In [7]:
plotInteractive(0.0, 50, 'ReH')

In [8]:
plotInteractive(0.3, 50, 'ReH')

In [3]:
errors = [0.0, 0.01, 0.025, 0.05, 0.1, 0.15, 0.25, 0.30]
for err in errors: 
    plotInteractive(err, 403, 'ReH', show = False)
    plotInteractive(err, 403, 'ReE', show = False)
    plotInteractive(err, 403, 'ReHtilde', show = False)